# The Guardian Opinions: Text Classification and Bag-of-Words

Outline:
- Import and explore data
- Apply text preprocessing techniques
- -Implement the bag-of-words model

## Import and explore data

The following data was collected using a Scrapy spider, found in this repository under /guardianscraper/guardianscraper/spiders/guardianspider.py

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./guardian_data.csv')
df.head(10)

,date,title,author,article
0,2023/feb/07,Liz Truss set out her grand plan to me – and i...,Katy Balls,"“You’ve set the cat among the pigeons,” messag..."
1,2023/feb/07,The Gareth Thomas case proves it: no one wins ...,James Greig,Few public figures alive today have done more ...
2,2023/feb/07,There’s no cycle of violence in Jerusalem – on...,Jalal Abukhater,"Almost every day, the bulldozers are on the mo..."
3,2023/feb/06,The ‘leftwing economic establishment’ did not ...,Polly Toynbee,"“This soul-searching has not been easy,” she w..."
4,2023/feb/07,I have seen race hate in the US and UK and the...,Al Sharpton,I came to London more than 30 years ago to pro...
5,2023/feb/07,As the detective who inspired TV’s Prime Suspe...,Jackie Malton,We have now heard for the first time from Davi...
6,2023/feb/07,"Britain, we had a thing with Truss and Johnson...",Marina Hyde,Liz Truss is now eluded by two major types of ...
7,2023/feb/07,Would I ditch a date after 51 minutes? I’ve wa...,Elle Hunt,Fifty-one minutes. It’s too long for a meeting...
8,2023/feb/07,"As an ex police officer, this much is clear: a...",Steve White,The sentencing of David Carrick and the dreadf...
9,2023/feb/07,Jeremy Hunt says focus on the ‘economically in...,Frances Ryan,"As the government lurches between screw-ups, s..."


In [6]:
sample_article = df.article.values[0]
